In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

Loader PDF

In [2]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("data/wilee.pdf")
pages = loader.load()

Splitting

In [3]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
splits = text_splitter.split_documents(pages)

Embedding

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

Vectorstores

In [5]:
from langchain.vectorstores import Chroma

persist_directory = 'data/chroma'

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

# vectordb = Chroma(persist_directory=persist_directory, 
#                     embedding_function=embedding)

working with metadata using self-query retriever

In [6]:
from langchain.chat_models import ChatOpenAI
llm_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [7]:
# from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain

In [8]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser

condense_q_system_prompt = """Given a chat history and the latest user question \
which might reference the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
condense_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
condense_q_chain = condense_q_prompt | llm | StrOutputParser()


In [9]:
# Run chain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key='chat_history', 
    return_messages=True, 
    output_key='answer')

retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    memory=memory,
    retriever=retriever, 
    qaChainOptions: {
        type: "stuff", 
        prompt: QA_PROMPT}
    return_source_documents=True
)

In [10]:
for i in range(3):
    question = input("enter ")
    result = qa({"question": question})
    print(result['answer'])
    print('---------------------------------')

If x = 5 and y = 9, then z can be calculated by substituting these values into the equation x + y = z.

So, z = 5 + 9 = 14.
---------------------------------
I'm sorry, but I don't have enough information to determine the answer to the previous question. Could you please provide more context or clarify your question?
---------------------------------


KeyboardInterrupt: 